In [4]:
import logging
# Some packages used by pvtrace are a little noisy
logging.getLogger('trimesh').disabled = True
logging.getLogger('shapely.geos').disabled = True
from ipywidgets import interact
from pvtrace import *
import numpy as np



In [5]:
def interact_ray(scene, vis):
    ray_ids = []

    def move_ray(x=0.0, y=0.0, z=0.0, theta=0.0, phi=0.0, nanometers=555.0):
        # Clear old objects
        [vis.remove_object(_id) for _id in ray_ids]

        # Create new array with position from the interact UI
        # ray = Ray(
        #     position=(x, y, z),
        #     direction=(
        #         np.sin(np.radians(theta)) * np.cos(np.radians(phi)),
        #         np.sin(np.radians(theta)) * np.sin(np.radians(phi)),
        #         np.cos(np.radians(theta))
        #     ),
        #     wavelength=nanometers
        # )

        # Re-create the scene with the new ray but reuse the renderers
        renderer = MeshcatRenderer(wireframe=True, open_browser=True)
        renderer.render(scene)
        for ray in scene.emit(60):
            steps = photon_tracer.follow(scene, ray, maxsteps=10)
            path, events = zip(*steps)
            vis.render(scene)
            renderer.add_ray_path(path)
            # time.sleep(0.1)

        # steps = photon_tracer.follow(scene, ray, maxsteps=100)
        # path, events = zip(*steps)
        # vis.render(scene)
        #
        # # Remove old rays; add new rays
        # ray_ids.clear()
        # ray_ids.extend(vis.add_ray_path(path))

    return interact(
        move_ray,
        x=(-5, 5, 0.01),
        y=(-5, 5, 0.01),
        z=(-5, 5, 0.01),
        theta=(0, 180, 1),
        phi=(0, 360, 1),
        nanometers=(300, 800, 1)
    )


def interact_beam(scene, vis):
    ray_ids = []

    def move_ray(x=0.0, y=0.0, z=0.0, theta=0.0, phi=0.0, nanometers=555.0):
        # Clear old objects
        [vis.remove_object(_id) for _id in ray_ids]

        # Create new array with position from the interact UI
        ray = Ray(
            position=(x, y, z),
            direction=(
                np.sin(np.radians(theta)) * np.cos(np.radians(phi)),
                np.sin(np.radians(theta)) * np.sin(np.radians(phi)),
                np.cos(np.radians(theta))
            ),
            wavelength=nanometers
        )

        # Re-create the scene with the new ray but reuse the renderers
        steps = photon_tracer.follow(scene, ray, maxsteps=50)
        path, events = zip(*steps)
        vis.render(scene)

        # Remove old rays; add new rays
        ray_ids.clear()
        ray_ids.extend(vis.add_ray_path(path))

    return interact(
        move_ray,
        x=(-5, 5, 0.01),
        y=(-5, 5, 0.01),
        z=(-5, 5, 0.01),
        theta=(0, 180, 1),
        phi=(0, 360, 1),
        nanometers=(300, 800, 1)
    )

In [6]:
import functools
def pv_sphere_test():
    world = Node(
        name="world (air)",
        geometry=Sphere(
            radius=10.0,
            material=Material(refractive_index=1.0),
        )
    )
    sphere1 = Node(
        name="sphere1 (glass)",
        geometry=Sphere(
            radius=8.0,
            material=Material(refractive_index=100,
                              ),

        ),
        parent=world
    )
    # components=[
    #     Scatterer(
    #         coefficient=1.0,  # e.g. units: cm-1
    #         quantum_yield=1.0
    #    )
    # ]
    sphere1.translate((0.0, 0.0, 0.0))
    a = 0.4
    x_ar = np.linspace(-5*a, 5*a, 11)
    y_ar = [-3*a, -2*a,-a, 0, a, 2*a, 3*a]
    z_ar = [-3*a, -2*a,-a, 0, a, 2*a, 3*a]
    for x in x_ar:
        for y in y_ar:
            for z in z_ar:
                sphere2 = Node(
                    name=f"sphere: {x}, {y}, {z} (glass)",
                    geometry=Sphere(
                        radius=a/2,
                        material=Material(refractive_index=1.5),
                    ),
                    parent=world

                )
                sphere2.translate((x, y, z))
    light = Node(
    name="Light (555nm)",
    light=Light(direction=functools.partial(cone, np.pi/32)),
    parent=world
)
    light.translate((0.0, 0.0, -7.95))
    scene = Scene(world)

    renderer = MeshcatRenderer(wireframe=True)
    renderer.render(scene)
    renderer.vis.jupyter_cell()

    _ = interact_ray(scene, renderer)


pv_sphere_test()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


interactive(children=(FloatSlider(value=0.0, description='x', max=5.0, min=-5.0, step=0.01), FloatSlider(value…